In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-11-30 2021-12-01


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|████████████▌                                                                 | 81/505 [01:17<06:53,  1.02it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|██████████████████████████████████████████████▋                              | 306/505 [04:52<02:46,  1.20it/s]

project_market prices documents must be a non-empty list


 99%|████████████████████████████████████████████████████████████████████████████▌| 502/505 [07:25<00:02,  1.34it/s]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|████████████████████████████████████████████████████████████████████████████▊| 504/505 [07:26<00:00,  1.40it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████| 505/505 [07:27<00:00,  1.13it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|██████████                                                                    | 65/505 [02:02<13:50,  1.89s/it]

'date'


 16%|████████████▏                                                                 | 79/505 [02:29<13:17,  1.87s/it]

'date'


 99%|████████████████████████████████████████████████████████████████████████████▌| 502/505 [15:55<00:05,  1.88s/it]

...

'date'


100%|████████████████████████████████████████████████████████████████████████████▊| 504/505 [15:59<00:01,  1.87s/it]

'date'


100%|█████████████████████████████████████████████████████████████████████████████| 505/505 [16:01<00:00,  1.90s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
44,0.283341,0.158780,0.111308,0.031578,0.031578,AAPL,Apple,Information Technology
368,0.503798,0.251572,0.242160,0.025382,0.025382,PFE,Pfizer,Health Care
78,0.365318,0.000000,0.000000,0.008613,0.022518,COG,Cabot Oil & Gas,Energy
438,0.568659,0.476690,-0.052814,0.006834,0.006834,TSLA,Tesla,Consumer Discretionary
254,-0.247560,0.019054,0.005696,0.006189,0.006189,IPGP,IPG Photonics,Information Technology
65,0.309321,0.015204,-0.039482,0.004856,0.004856,BIO,Bio-Rad Laboratories,Health Care
138,-0.088466,-0.174877,-0.161390,0.001027,0.001027,XRAY,Dentsply Sirona,Health Care
307,-0.004547,-0.079730,-0.147199,0.000267,0.000267,MRK,Merck & Co.,Health Care
303,0.178743,-0.000485,0.000000,0.000000,0.000000,MXIM,Maxim Integrated,Information Technology
296,-0.370778,-0.155881,-0.135455,-0.000765,-0.000765,MKTX,MarketAxess,Financials


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Information Technology,0.236952,0.046992,-0.000670,-0.020195,-0.020195
Health Care,0.150589,-0.032221,-0.053829,-0.023008,-0.023008
Real Estate,0.412458,0.038069,-0.016582,-0.024012,-0.024012
Energy,0.554691,-0.001605,-0.085412,-0.025082,-0.024450
Consumer Discretionary,0.246501,0.024903,-0.036948,-0.025699,-0.025699
Industrials,0.229341,0.019262,-0.036652,-0.027112,-0.027112
Consumer Staples,0.048512,-0.003555,-0.017050,-0.027664,-0.027664
Financials,0.324592,-0.009030,-0.060188,-0.028028,-0.028028
Utilities,0.083594,0.012027,-0.029352,-0.029842,-0.029842
